In [ ]:
import cdflib
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import datetime as dt
from omni_seg_3 import omni_seg
from gipm_transform_coeffs import gipm_transform_coeffs
from new_xyz import new_xyz
import glob
from math import pi
import math
from XMA_finder import XMA_finder
from gipm_locs_quick import gipm_locs_quick


In [ ]:
#open OMNI CSVs
list_all = []

path = "/Users/apx059/Documents/Location Only Checks/OMNI Raw/**"

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders
om_csvs = []

for element in list_all:
    if '.csv' in element:
        om_csvs.append(element)
        
om_dfs = []

for element in om_csvs:
    om = pd.read_csv(element)
    om_dfs.append(om)
    
omni_all = pd.concat(om_dfs)
omni_all['datetime'] = pd.to_datetime(omni_all['datetime'])

omni_all = omni_all.set_index('datetime')

for element in om_dfs:
    element['datetime'] = pd.to_datetime(element['datetime'])
    element = element.set_index('datetime', inplace = True)


In [ ]:
##now to produce OMNI histograms of conditions
IMF_B = omni_all['B_mag']
SW_V = omni_all['V_gse']
N_p = omni_all['Np']
M_A = omni_all['M_A']
Cone_A = omni_all['cone angle']

fig = plt.figure(figsize=(6, 9), tight_layout=True)
gs = gridspec.GridSpec(3, 2)

ax = fig.add_subplot(gs[0,0])
#bins = [0,10,20,30,40,50,60,70,80,90]
ax.hist(IMF_B, bins = [0,1,2,4,6,8,10,12,14,16,18,22,26,30,40,50,60,70], edgecolor='black', linewidth=1.2, density = False)
ax.set_xlabel("IMF B, nT (OMNI)")
ax.set_ylabel("No. Observations")
ax.set_xscale('log')
ax.set_yscale('log')

fig.suptitle('Solar Wind Conditions Feb 2001- Feb 2024')

ax = fig.add_subplot(gs[0,1])
ax.hist(SW_V, edgecolor='black', linewidth=1.2, density = False)
ax.set_xlabel("Solar Wind Speed, km/s (OMNI)")
ax.set_ylabel("No. Observations")
#ax.set_xscale('log')
ax.set_yscale('log')

ax = fig.add_subplot(gs[1,0])
ax.hist(M_A, bins = [1,2,3,4,5,6,7,8,10,12,14,16,18,20,25,30,40,50,70,100,200,400], edgecolor='black', linewidth=1.2, density = False)
ax.set_xlabel("Alfven Mach Number(OMNI)")
ax.set_ylabel("No. Observations")
ax.set_xscale('log')
ax.set_yscale('log')

ax = fig.add_subplot(gs[1,1])
#bins = [0,10,20,30,40,50,60,70,80,90]
ax.hist(N_p, bins = [0,1,2,4,6,8,10,12,14,16,18,22,26,30,40,50,60,70], edgecolor='black', linewidth=1.2, density = False)
ax.set_xlabel("Proton Density, /CC (OMNI)")
ax.set_ylabel("No. Observations")
ax.set_xscale('log')
ax.set_yscale('log')

ax = fig.add_subplot(gs[2,1])
#bins = [0,10,20,30,40,50,60,70,80,90]
ax.hist(Cone_A,bins = [0,10,20,30,40,50,60,70,80,90], edgecolor='black', linewidth=1.2, density = False)
ax.set_xlabel("IMF Cone Angle (OMNI)")
ax.set_ylabel("No. Observations")
#ax.set_xscale('log')
ax.set_yscale('log')

#fig.show()


In [ ]:
%%time
##load Cluster CSVs

list_all = []

path = "/Users/apx059/Documents/Location Only Checks/Cluster Raw/**"

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders
cl_file_list = []

for element in list_all:
    if '.csv' in element:
        cl_file_list.append(element)

cl_dfs = []

for file in cl_file_list:
    df = pd.read_csv(file,encoding='utf-8')
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace = True)
    cl_dfs.append(df)

In [ ]:
#need to reorder bc they're not automatically in the right order
reordered_om_df = []

for df in cl_df_list:
    fd_list = df.index
    fd = fd_list[0]
    first_read = fd.strftime("%Y")
    for omni_temp in om_dfs:
        fd_om_list_temp = omni_temp.index
        fd_om_temp = fd_om_list_temp[0]
        first_read_om_t = fd_om_temp.strftime("%Y")
        if first_read_om_t == first_read:
            reordered_om_df.append(omni_temp)

In [ ]:
%%time
#break into intervals with averaged OMNI values. for subsequent GIPM plotting
#first, pair Cluster and OMNI dataframes

om_ave_list = []
dayside_cl_list = []

for df, om_df in zip(cl_df_list,reordered_om_df):
    #mask to just dayside
    cluster_df_day = df.loc[(df['X_gse']> 0)]
    datetime_list = cluster_df_day.index
    om_averages = omni_seg(om_df, datetime_list)
    om_ave_list.append(om_averages)
    dayside_cl_list.append(cluster_df_day)
    

In [ ]:
%%time

for om_ave_df in om_ave_list:
    CSV_path = '/Users/apx059/Documents/Location Only Checks/OMNI Averages'
    start_ind = om_ave_df.index[0]
    start_year = start_ind.strftime("%Y")
    start_year_int = int(start_year)
    end_year_int = start_year_int + 1
    end_year = str(end_year_int)
    CSV_file_name = CSV_path + 'OMNI' + '_Feb' + start_year + '_Feb' + end_year + '.csv'
    om_ave_df.to_csv(CSV_file_name)

df_no = 0
for cl_day in dayside_cl_list:
    CSV_path = '/Users/apx059/Documents/Location Only Checks/Cluster Dayside Only'
    df_ref = str(df_no)
    CSV_file_name = CSV_path + 'Cluster Dayside' + '_dfref' + df_ref +'.csv'
    cl_day.to_csv(CSV_file_name)
    df_no = df_no + 1


In [ ]:
temp_om_ave_list = om_ave_list[0:9]
print(len(temp_om_ave_list))

In [ ]:
for i in temp_om_ave_list:
    i = i.reset_index(inplace = True)

In [ ]:
temp_om_ave_list[0]

In [ ]:
%%time
#find GIPM rotation matrices and scaling coefficient for every Cluster location
GIPM_mat_list = []
FAC_coeff_list = []

for om_df in om_ave_list:
    GIPM_mat, FAC_coeffs = gipm_transform(om_df)
    GIPM_mat_list.append(GIPM_mat)
    FAC_coeff_list.append(FAC_coeffs)


In [ ]:

#now transform all locations to GIPM

def gipm_locs_quick(df_cluster, GIPM_matrices, FAC_coeffs):
    
    Cluster_GIPM_orbits = []
    df_cluster_ind = df_cluster.index
    #mask to just the two minute window in cluster data and find mean location
    
    for i,k in zip(df_cluster_ind, FAC_coeffs):
        m = FAC_coeffs.index(k)
        
        x_gse = df_cluster.loc[i,'X_gse']
        y_gse = df_cluster.loc[i,'Y_gse']
        z_gse = df_cluster.loc[i,'Z_gse']
        
        #form location arrays for every observation and then transform using coeffs/FAC
        Cluster_GSE = np.array([x_gse, y_gse, z_gse])
        #rotate
        trans_mat = GIPM_matrices[m]
        Cluster_GIPM = new_xyz(Cluster_GSE, trans_mat)
        #FAC scaling
        #r is scaled by this but the direction should remain the same
        x_a = Cluster_GIPM[0]
        y_a = Cluster_GIPM[1]
        z_a = Cluster_GIPM[2]
        r_a = (x_a**2 + y_a**2 + z_a**2)**0.5
        #find original angles
        theta_a = np.arccos(z_a/r_a)
        phi_a = np.arctan(y_a/x_a)

        r_a = r_a*k

        #new x_a, y_a, z_a
        x_a = r_a*(np.sin(theta_a))*(np.cos(phi_a))
        y_a = r_a*(np.sin(theta_a))*(np.sin(phi_a))
        z_a = r_a*(np.cos(theta_a))
                
        Cluster_GIPM = np.array([x_a, y_a, z_a])
                
        #append result
        Cluster_GIPM_orbits.append(Cluster_GIPM)
        
    Cluster_dt_loc = pd.DataFrame({'datetime':i, 'GIPM Loc': Cluster_GIPM_orbits})
        

    return(Cluster_dt_loc)  
    

    


In [ ]:
%%time

Cluster_GIPM_locs_list = []

for i,j,k in zip(cl_dfs, GIPM_mat_list, FAC_coeff_list):
    Cluster_dt_loc = gipm_locs_quick(i, j, k)
    Cluster_GIPM_locs_list.append(Cluster_dt_loc)

In [ ]:
# len(Cluster_GIPM_locs_list) is 61

rest_cl_dfs = cl_dfs[61:89]
rest_GIPM_mat = GIPM_mat_list[61:89]
rest_FAC_coeffs = FAC_coeff_list[61:89]

for i,j,k in zip(rest_cl_dfs, rest_GIPM_mat, rest_FAC_coeffs):
    Cluster_dt_loc = gipm_locs_quick(i, j, k)
    Cluster_GIPM_locs_list.append(Cluster_dt_loc)

In [ ]:
%%time

##turn ONE df into the correct format

df = Cluster_GIPM_locs_list[0]

GIPM_temp_loc = []

for i in df['GIPM Loc']:
    
    GIPM_temp_loc.append(i)

    X_list_1 = []
    Y_list_1 = []
    Z_list_1 = []

    for i in GIPM_temp_loc:
        X_list_1.append(i[0])
        Y_list_1.append(i[1])
        Z_list_1.append(i[2])

dt_list = df['datetime'].to_list()

Cluster_GIPM_dict = {'window start': dt_list, 'Cluster Loc GIPM X': X_list_1, 'Cluster Loc GIPM Y': Y_list_1,'Cluster Loc GIPM Z': Z_list_1}     

Cluster_GIPM_df = pd.DataFrame(Cluster_GIPM_dict)

In [ ]:
%%time
#new dataframes with columns separated into X, Y, Z
GIPM_loc_dfs_new = []

for df in Cluster_GIPM_locs_list:
    GIPM_temp_loc = []
    
    for i in df['GIPM Loc']:
        GIPM_temp_loc.append(i)

        X_list_1 = []
        Y_list_1 = []
        Z_list_1 = []

        for i in GIPM_temp_loc:
            X_list_1.append(i[0])
            Y_list_1.append(i[1])
            Z_list_1.append(i[2])

    dt_list = df['datetime'].to_list()

    Cluster_GIPM_dict = {'window start': dt_list, 'Cluster Loc GIPM X': X_list_1, 'Cluster Loc GIPM Y': Y_list_1,'Cluster Loc GIPM Z': Z_list_1}     
    Cluster_GIPM_df = pd.DataFrame(Cluster_GIPM_dict)
    GIPM_loc_dfs_new.append(Cluster_GIPM_df)
    

In [ ]:
len(GIPM_loc_dfs_new)

In [ ]:
#now save all of these dfs to CSV
#NOTE there is dt error in these CSVs! The same date is repeated.
startref = 0

for df in GIPM_loc_dfs_new:
        CSV_path = '/Users/apx059/Documents/Location Only Checks/Cluster GIPM/'
        ref = str(startref)
        dt_df = df['window start']
        start_ind = dt_df[0]
        start_year = start_ind.strftime("%Y")
        start_year_int = int(start_year)
        end_year_int = start_year_int + 1
        end_year = str(end_year_int)
        CSV_file_name = CSV_path + '_Feb' + start_year + '_Feb' + end_year + 'df_' +'.csv'
        df.to_csv(CSV_file_name, encoding = 'utf-8')
        startref = startref + 1

In [ ]:
%%time
all_GIPM_locs = pd.concat(GIPM_loc_dfs_new, ignore_index = True)

In [ ]:
%%time

x_locs = all_GIPM_locs['Cluster Loc GIPM X']
y_locs = all_GIPM_locs['Cluster Loc GIPM Y']
z_locs = all_GIPM_locs['Cluster Loc GIPM Z']
##use numpy histogram to get actual bin numbers
x_bin_edges = range(30)
y_bin_edges = range(-30,30)
HistXY, xedg, yedg = np.histogram2d(x_locs, y_locs, bins=[x_bin_edges, y_bin_edges])
HistXY = HistXY.T

z_bin_edges = range(-30,30)
HistXZ, xedg, zedg = np.histogram2d(x_locs, z_locs, bins=[x_bin_edges, z_bin_edges])
HistXZ = HistXZ.T

HistXY[HistXY == 0] = np.nan
HistXZ[HistXZ == 0] = np.nan

In [ ]:
omni_ave_all = pd.concat(om_ave_list)
XMA_all = XMA_finder(omni_ave_all)


In [ ]:
#import modules
import pandas as pd
from XMA_finder import XMA_finder
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import cm
import matplotlib
from merka05_surface_eq_array_GIPM_2 import merka05_surface_eq_array_GIPM

x = np.linspace(0, 20, 100) #x coordinates (Re)
y = np.linspace(-30, 30, 100) #y coordinates (Re)
z = 0 #z coordinates in Re

[Xgipm,Ygipm,Zgipm] = np.meshgrid(x,y,z,indexing="ij")

fitting_coeffs = merka05_surface_eq_array_GIPM(XMA_all)

Xn = Xgipm
Yn = Ygipm
Zn = Zgipm
f = fitting_coeffs[0]*Xn**2 + fitting_coeffs[1]*Yn**2 + fitting_coeffs[2]*Zn**2+ 2*fitting_coeffs[3]*Xn*Yn + 2*fitting_coeffs[4]*Yn*Zn + 2*fitting_coeffs[5]*Xn*Zn + 2*fitting_coeffs[6]*Xn+2*fitting_coeffs[7]*Yn + 2*fitting_coeffs[8]*Zn + fitting_coeffs[9]

#tanfieldangle = y/x

tan_angle_0 = np.tan(np.deg2rad(15))
tan_angle_1 = np.tan(np.deg2rad(45))
tan_angle_2 = np.tan(np.deg2rad(75))

#magnetopause model, D = 2 nPa

m_1 = 10.22
m_2 = 1.29
m_3 = 0.184
m_4 = 8.14
m_5 = 6.6
m_6 = 0.58
m_7 = -0.007
m_8 = 0.024

#use B_z =0
B_z = 0
D_p = 2

alpha = (m_6 + m_7*B_z)*(1 +m_8*(np.log(D_p)))

tanh_angle = m_3*(B_z+m_4)
r_0 = (m_1 + m_2*np.tanh(tanh_angle))*(D_p**(-1/m_5))

pi = np.pi

theta = np.arange(-pi/2, pi/2, 0.01)

r_mod = (2/(1+np.cos(theta)))**alpha
r = r_0*r_mod

X_shue = r*(np.cos(theta))
R_shue = r*(np.sin(theta))


###################
fig, ax = plt.subplots()
#subfigs = fig.subfigures(1, 1)
#axsLeft = subfigs.subplots(1, 3, sharey=False)
#gs = gridspec.GridSpec(2, 2)
plt.suptitle('Cluster Locations All 9 Years')

ax.contour(Xgipm[:,:,0],Ygipm[:,:,0],f[:,:,0],levels = [0],colors="black",linewidths=1)
ax.plot(X_shue, R_shue, linewidth=1, color='k')

ax.set_aspect('equal')
ax.set_xlabel("$X_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")
ax.set_ylabel("$Y_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")


#want to find three points on the bow shock surface, at y=0 and y=±8 and draw lines from there
#without exceeding current bounds of plot
inter_med = fitting_coeffs[6]**2 - (fitting_coeffs[0]*fitting_coeffs[9])
X_BS_nose = (-fitting_coeffs[6] + np.sqrt(inter_med))/fitting_coeffs[0]

x_s = X_BS_nose
y_s = 0
x_e = 30
y_e = x_e*(-tan_angle_0)

#want to also have line for just solar wind flow along y=0

ax.hlines(y=0, xmin= 0, xmax=30, linewidth=1, color='k')
#ax.plot([x_s, x_e], [y_s, y_e], color='k',linewidth=1)
cmap = matplotlib.colormaps.get_cmap('Blues') 
cmap.set_bad(color='lightgrey')
plt.imshow(HistXY, interpolation='nearest', origin='lower', extent=[xedg[0], xedg[-1], yedg[0], yedg[-1]], vmax =1000, cmap = cmap)
#ax.set_ylim(-30,30)
#ax.set_xlim(0,30)
ax.invert_xaxis()
ax.invert_yaxis()
plt.colorbar()

plt.show()

In [ ]:
x = np.linspace(0, 20, 100) #x coordinates (Re)
y = np.linspace(-30, 30, 100) #y coordinates (Re)
z = 0 #z coordinates in Re

[Xgipm,Ygipm,Zgipm] = np.meshgrid(x,y,z,indexing="ij")

fitting_coeffs = merka05_surface_eq_array_GIPM(XMA_all)

Xn = Xgipm
Yn = Ygipm
Zn = Zgipm
f = fitting_coeffs[0]*Xn**2 + fitting_coeffs[1]*Yn**2 + fitting_coeffs[2]*Zn**2+ 2*fitting_coeffs[3]*Xn*Yn + 2*fitting_coeffs[4]*Yn*Zn + 2*fitting_coeffs[5]*Xn*Zn + 2*fitting_coeffs[6]*Xn+2*fitting_coeffs[7]*Yn + 2*fitting_coeffs[8]*Zn + fitting_coeffs[9]

x_1 = np.linspace(0, 20, 100) #x coordinates (Re)
y_1 = 0 #y coordinates (Re)
z_1 = np.linspace(-30, 30, 100) #z coordinates in Re

[Xgipm_1,Ygipm_1,Zgipm_1] = np.meshgrid(x_1,y_1,z_1,indexing="ij")
Xn_1 = Xgipm_1
Yn_1 = Ygipm_1
Zn_1 = Zgipm_1
f_1 = fitting_coeffs[0]*Xn_1**2 + fitting_coeffs[1]*Yn_1**2 + fitting_coeffs[2]*Zn_1**2+ 2*fitting_coeffs[3]*Xn_1*Yn_1 + 2*fitting_coeffs[4]*Yn_1*Zn_1 + 2*fitting_coeffs[5]*Xn_1*Zn_1 + 2*fitting_coeffs[6]*Xn_1+2*fitting_coeffs[7]*Yn_1 + 2*fitting_coeffs[8]*Zn_1 + fitting_coeffs[9]

#magnetopause model, D = 2 nPa

m_1 = 10.22
m_2 = 1.29
m_3 = 0.184
m_4 = 8.14
m_5 = 6.6
m_6 = 0.58
m_7 = -0.007
m_8 = 0.024

#use B_z =0
B_z = 0
D_p = 2

alpha = (m_6 + m_7*B_z)*(1 +m_8*(np.log(D_p)))

tanh_angle = m_3*(B_z+m_4)
r_0 = (m_1 + m_2*np.tanh(tanh_angle))*(D_p**(-1/m_5))

pi = np.pi

theta = np.arange(-pi/2, pi/2, 0.01)

r_mod = (2/(1+np.cos(theta)))**alpha
r = r_0*r_mod

X_shue = r*(np.cos(theta))
R_shue = r*(np.sin(theta))


###################
fig, ax = plt.subplots()
subfigs = fig.subfigures(1, 1)
axsLeft = subfigs.subplots(1, 2, sharey=False)
subfigs.suptitle('Cluster Locations All 9 Years')

ax0 = axsLeft[0]

ax0.contour(Xgipm[:,:,0],Ygipm[:,:,0],f[:,:,0],levels = [0],colors="black",linewidths=1)
ax0.plot(X_shue, R_shue, linewidth=1, color='k')

ax0.set_aspect('equal')
ax0.set_xlabel("$X_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")
ax0.set_ylabel("$Y_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")


#want to find three points on the bow shock surface, at y=0 and y=±8 and draw lines from there
#without exceeding current bounds of plot
inter_med = fitting_coeffs[6]**2 - (fitting_coeffs[0]*fitting_coeffs[9])
X_BS_nose = (-fitting_coeffs[6] + np.sqrt(inter_med))/fitting_coeffs[0]

x_s = X_BS_nose
y_s = 0
x_e = 30
y_e = x_e*(-tan_angle_0)

#want to also have line for just solar wind flow along y=0

ax0.hlines(y=0, xmin= 0, xmax=30, linewidth=1, color='k')
#ax.plot([x_s, x_e], [y_s, y_e], color='k',linewidth=1)
cmap = matplotlib.colormaps.get_cmap('Blues') 
cmap.set_bad(color='lightgrey')
im = ax0.imshow(HistXY, interpolation='nearest', origin='lower', extent=[xedg[0], xedg[-1], yedg[0], yedg[-1]], vmax =5000, cmap = cmap)
#ax.set_ylim(-30,30)
#ax.set_xlim(0,30)
ax0.invert_xaxis()
ax0.invert_yaxis()
fig.colorbar(mappable=im,location='bottom',anchor=(0.5, 0), panchor=(0.5, 0.2), pad=0.1, ax=axsLeft, label='No. of Observations')

ax1 = axsLeft[1]

ax1.contour(Xgipm_1[:,0,:],Zgipm_1[:,0,:],f_1[:,0,:],levels = [0],colors="black",linewidths=1)
ax1.plot(X_shue, R_shue, linewidth=1, color='k')

ax1.set_aspect('equal')
ax1.set_xlabel("$X_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")
ax1.set_ylabel("$Z_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")


ax1.hlines(y=0, xmin= 0, xmax=30, linewidth=1, color='k')
#ax.plot([x_s, x_e], [y_s, y_e], color='k',linewidth=1)
ax1.imshow(HistXZ, interpolation='nearest', origin='lower', extent=[xedg[0], xedg[-1], yedg[0], yedg[-1]], vmax =5000, cmap = cmap)
#ax.set_ylim(-30,30)
#ax.set_xlim(0,30)
ax1.invert_xaxis()
ax1.invert_yaxis()
#fig.show()